In [0]:
!pip install sparkxgb

  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.5 MB/s eta 0:00:00
  Created wheel for sparkxgb: filename=sparkxgb-0.1-py3-none-any.whl size=5630 sha256=88dbd3d1a94c52280a7e9860bf1e340c4a609f2575f40ec0e486b028be358cef
  Stored in directory: /root/.cache/pip/wheels/b7/0c/a1/786408e13056fabeb8a72134e101b1e142fc95905c7b0e2a71
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767590 sha256=ef70bbf13e1bdc16f6fb526112234a1cf0e1ada18ecb87f7cdd0ec223a527b04
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built sparkxgb pyspark
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
data

In [0]:
#imports and paths

# General Packages:
import numpy as np
import pandas as pd
import re
import time
from functools import reduce
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.gridspec as gridspec
import seaborn as sns
from datetime import date, timedelta
from pyspark.sql.functions import col, isnan, when, mean,avg, stddev, count, sum, countDistinct, min, round, max, year
from pyspark.ml.stat import Correlation
from pyspark.sql import functions as F
from collections import namedtuple

# For Pipeline
from pyspark.ml import Transformer
from pyspark.sql import DataFrame, Row
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.sql.functions import array, create_map, struct

# For Feature Engineering
from pyspark.ml.feature import VectorAssembler,MinMaxScaler,BucketedRandomProjectionLSH,VectorSlicer,IndexToString
from pyspark.sql.types import FloatType, IntegerType, TimestampType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import col, to_timestamp, hour, concat, date_format, rand, when,concat,substring,lit,udf,lower,sum as ps_sum,count as ps_count,row_number
from pyspark.sql.window import *
from pyspark.sql.window import Window
from graphframes import GraphFrame

# For Modeling:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier
from sklearn.ensemble import AdaBoostClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
from sparkxgb import XGBoostClassifier
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import random

#dbutils.fs.help()
data_BASE_DIR = "/mnt/"
#display(dbutils.fs.ls(f"{data_BASE_DIR}"))

In [0]:
blob_container  = "w261-container"
storage_account = "whiskeythedog"
secret_scope    = "whiskeythedog-scope"                   
secret_key      = "whikeythedog-key"  
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
# Inspect the Mount's Final Project folder 
# Please IGNORE dbutils.fs.cp("/mnt/mids-w261/datasets_final_project/stations_data/", "/mnt/mids-w261/datasets_final_project_2022/stations_data/", recurse=True)
mids261_mount_path = "dbfs:/mnt/mids-w261/" #no permission to write to this mounted blob storage; 
display(dbutils.fs.ls(f"{mids261_mount_path}")) #3M, 12M, 36M, 60M

path,name,size,modificationTime
dbfs:/mnt/mids-w261/HW5/,HW5/,0,0
dbfs:/mnt/mids-w261/OTPW_12M/,OTPW_12M/,0,0
dbfs:/mnt/mids-w261/OTPW_1D_CSV/,OTPW_1D_CSV/,0,0
dbfs:/mnt/mids-w261/OTPW_36M/,OTPW_36M/,0,0
dbfs:/mnt/mids-w261/OTPW_3M/,OTPW_3M/,0,0
dbfs:/mnt/mids-w261/OTPW_3M_2015.csv,OTPW_3M_2015.csv,1500620247,1679772070000
dbfs:/mnt/mids-w261/OTPW_60M/,OTPW_60M/,0,0
dbfs:/mnt/mids-w261/airport-codes_csv.csv,airport-codes_csv.csv,6232459,1677623514000
dbfs:/mnt/mids-w261/datasets_final_project/,datasets_final_project/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/,datasets_final_project_2022/,0,0


In [0]:
# Inspect the Mount's Final Project folder 
# Please IGNORE dbutils.fs.cp("/mnt/mids-w261/datasets_final_project/stations_data/", "/mnt/mids-w261/datasets_final_project_2022/stations_data/", recurse=True)
data_BASE_DIR = "dbfs:/mnt/mids-w261/datasets_final_project_2022/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))
#display(dbutils.fs.ls(f"{data_BASE_DIR}stations_data/"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/,parquet_airlines_data/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/,parquet_airlines_data_1y/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/,parquet_weather_data/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_1y/,parquet_weather_data_1y/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m/,parquet_weather_data_3m/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_6m/,parquet_weather_data_6m/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/,stations_data/,0,0


# Data Ingestion

In [0]:
#Load the 60 month joined dataset
path_60m =[f"{mids261_mount_path}OTPW_60M"]
OTPW_60M = spark.read.option("header",True) \
                       .option("compression", "gzip") \
                       .option("delimiter",",") \
                       .csv(path_60m).cache()
OTPW_60M.createOrReplaceTempView("OTPW_60M")

# Custom Transformation Functions

In [0]:
class CustomFunctionTransformer(Transformer):
    def __init__(self, custom_function):
        self.custom_function = custom_function
    
    def _transform(self, df: DataFrame) -> DataFrame:
        # Apply your custom function to the DataFrame
        transformed_data = self.custom_function(df)
        return transformed_data

In [0]:
def data_cleansing(df):

    # Drop duplicates
    df = df.dropDuplicates()

    # Feature Selection
    short  = ['QUARTER','DAY_OF_MONTH','DAY_OF_WEEK','YEAR','MONTH','FL_DATE','OP_UNIQUE_CARRIER',
    'ORIGIN',
    'pagerank',
    'inDegree',
    'outDegree',
    'DEST',
    'DEP_TIME',
    'DEP_DEL15',
    'DISTANCE',
    'origin_type',
    'sched_depart_date_time_UTC',
    'HourlyDewPointTemperature',
    'HourlyDryBulbTemperature',
    'HourlyPrecipitation',
    'HourlyRelativeHumidity',
    'HourlySeaLevelPressure',
    'HourlyVisibility',
    'HourlyWetBulbTemperature',
    'HourlyWindDirection',
    'HourlyWindSpeed']
    df = df.select(short).persist()

    # Cast numeric features to float 
    list_of_columns = ['DEP_DEL15', 'DISTANCE', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature','HourlyPrecipitation','HourlyRelativeHumidity','HourlySeaLevelPressure','pagerank',
    'inDegree',
    'outDegree']
    for column_name in list_of_columns:
        df = df.withColumn(column_name, df[column_name].cast(FloatType()))

    list_of_columns = ['QUARTER','DAY_OF_MONTH','DAY_OF_WEEK', 'YEAR', 'MONTH']
    for column_name in list_of_columns:
        df = df.withColumn(column_name, df[column_name].cast(StringType()))

    # Convert FL_Date to date:
    data_col = ['FL_DATE']
    for column_name in data_col:
        df = df.withColumn(column_name, date_format(df[column_name], "yyyy-MM-dd"))

    # Create hour_of_date column
    df = df.withColumn("hour_of_day", hour(col("sched_depart_date_time_UTC")))
    
    return df


def get_mode(df, column_name):
    mode_value = df.groupBy(column_name).count().orderBy(F.desc('count')).limit(1).collect()[0][0]
    return mode_value

# handle column and do imputation
def summarize_column(df, column_name, outcome_col, sample_rate=1, null_method='del', to_show=True, seed=42):
    df = df if sample_rate==1 else df.sample(False, sample_rate, seed)
    # cast the specific column to float
    df_with_cast = df.withColumn(column_name, col(column_name).cast('float'))
    df_filtered = df_with_cast.filter(col(column_name).isNotNull())
    
    # determine flags for different conditions
    df_with_flags = df_with_cast.withColumn('is_null_or_nan', col(column_name).isNull() | isnan(col(column_name)))
    df_with_flags = df_with_flags.withColumn('is_string_nan_or_null', col(column_name).isin('nan', 'null'))
    df_with_flags = df_with_flags.withColumn('is_float_castable', ~col('is_null_or_nan') & ~col('is_string_nan_or_null'))

    agg_result = df_with_flags.agg(
        count(col(column_name)).alias('count'),
        sum(when(col('is_null_or_nan'), 1).otherwise(0)).alias('nulls_nans_count'),
        sum(when(col('is_string_nan_or_null'), 1).otherwise(0)).alias('string_nans_nulls_count'),
        sum(when(col('is_float_castable'), 1).otherwise(0)).alias('float_castable_count'),
        sum(when(~col('is_float_castable'), 1).otherwise(0)).alias('uncastable_count'), # Assuming is_float_castable is a boolean column
        min(col(column_name)).alias('min_value'),
        max(col(column_name)).alias('max_value'),
        round(avg(col(column_name)), 2).alias('mean'),
        round(stddev(col(column_name)), 2).alias('std_dev')
    )

    # summarization values
    col_mean = df_filtered.agg(mean(col(column_name)).alias('mean')).collect()[0]['mean']
    col_mode = get_mode(df_filtered, column_name)
    col_median = df_filtered.approxQuantile(column_name, [0.5], 0.01)[0]
    print(col_mean, col_mode, col_median)

    # plot histogram if to_show is True
    if to_show:
        display(agg_result.collect())

        pandas_df = df_with_cast.select([column_name, outcome_col]).toPandas()

        plt.figure(figsize=(15, 6))
        plt.subplot(1, 3, (1,2))

        sns.histplot(data=pandas_df, x=column_name,  hue=outcome_col, bins=30, palette=['teal', 'rebeccapurple'],
                            kde=False, linewidth=0.5, multiple="stack", element="step", edgecolor="none")

        col_mean = df_filtered.agg(mean(col(column_name)).alias('mean')).collect()[0]['mean']
        col_median = df_filtered.approxQuantile(column_name, [0.5], 0.01)[0]
        plt.axvline(x=col_mean,   color='red',    linestyle='--')
        plt.axvline(x=col_median, color='pink',   linestyle='--')
        plt.gca().set_facecolor('lightgray')
        plt.title(column_name)

        plt.subplot(1, 3, 3)

        res = df_with_flags.withColumn(f'{column_name}_0_null', when((col(column_name).isNull()) & (col(outcome_col) == 0), 1) .otherwise(0)) \
                        .withColumn(f'{column_name}_0_num', when((~col(column_name).isNull()) & (col(outcome_col) == 0), 1) .otherwise(0)) \
                        .withColumn(f'{column_name}_1_null', when((col(column_name).isNull()) & (col(outcome_col) == 1), 1) .otherwise(0)) \
                        .withColumn(f'{column_name}_1_num', when((~col(column_name).isNull()) & (col(outcome_col) == 1), 1) .otherwise(0))

        aggy = res.agg(
                        count(f'{column_name}_0_null').alias('count'),
                        sum(f'{column_name}_0_null').alias('0_null'),
                        sum(f'{column_name}_0_num').alias('0_num'),
                        sum(f'{column_name}_1_null').alias('1_null'),
                        sum(f'{column_name}_1_num').alias('1_num')
                ).collect()

        display(aggy)

        bar_data = pd.DataFrame({
            'Category': ['Float-Castable', 'Float-Castable', 'Uncastable', 'Uncastable'],
            'Outcome': [0, 1, 0, 1],
            'Count': [aggy[0][2], aggy[0][4], aggy[0][1], aggy[0][3]]
        })

        plt.gca().set_facecolor('lightgray')
        sns.barplot(x='Category', y='Count', hue='Outcome', data=bar_data, palette=['teal', 'rebeccapurple'])
        plt.title('Null Counts by Category')

        plt.show()

    # return only the castable ones
    return agg_result

def summarize_columns(df, cols_to_summarize, view=True, seed=42):
    # summary results dataframe
    result_df = None

    # loop through the columns
    for col_name in cols_to_summarize:
        agg_result = summarize_column(df, col_name)
        result_df = result_df.union(agg_result) if result_df is not None else agg_result

    if view: 
        display(result_df)
    return result_df

outcome_col = 'DEP_DEL15'
cols_to_summarize = ['HourlyDewPointTemperature',
                'HourlyDryBulbTemperature',
                'HourlyPrecipitation',
                'HourlyRelativeHumidity',
                'HourlySeaLevelPressure',
                'HourlyVisibility',
                'HourlyWetBulbTemperature',
                'HourlyWindDirection',
                'HourlyWindGustSpeed',
                'HourlyWindSpeed',
                'pagerank',
                'inDegree',
                'outDegree']


def handle_missing_values(df, column_name, method='mean'):
    # cast the column to float
    df_casted = df.withColumn(column_name, col(column_name).cast('float'))
    
    # filter out the non-castable rows
    df_filtered = df_casted.filter(col(column_name).isNotNull())
    
    # determine the replacement value based on the method
    if method == 'mean':
        repl_value = df_filtered.agg(mean(col(column_name))).collect()[0][0]
    elif method == 'median':
        repl_value = df_filtered.approxQuantile(column_name, [0.5], 0)[0]
    elif method == 'mode':
        mode_count = df_filtered.groupBy(column_name).count().orderBy(F.desc('count'))
        repl_value = mode_count.collect()[0][0]

    # Fill the missing values with the replacement value
    df_filled = df_casted.withColumn(
        column_name, when(col(column_name).isNotNull(), col(column_name)).otherwise(repl_value)
    )
    
    return df_filled

def null_handling(df):
    null_cols =  [('HourlyDewPointTemperature', 'median'),
                    ('HourlyDryBulbTemperature', 'median'),
                    ('HourlyPrecipitation', 'median'),
                    ('HourlyRelativeHumidity', 'median'),
                    ('HourlySeaLevelPressure', 'median'),
                    ('HourlyVisibility', 'median'),
                    ('HourlyWetBulbTemperature', 'median'),
                    ('HourlyWindDirection', 'median'),
                    ('HourlyWindSpeed', 'median'),
                    ('pagerank', 'median'),
                    ('inDegree', 'median'),
                    ('outDegree', 'median'),]

    for column_name in null_cols:
        print(column_name)
        if isinstance(column_name, tuple): # if it's a tuple, then use the 1st element as method
            df = handle_missing_values(df, column_name[0], method=column_name[1])
        else:
            df = df.withColumn(column_name, when(col(column_name) == "null", 0).otherwise(col(column_name).cast("float")))

    # Drop NA rows
    df = df.na.drop()
    return df

def feature_engineering(df):
    ##### Create new features:

    ### Is Holiday
    holiday_list = ['2015-01-01','2015-01-02','2015-01-07','2015-01-08','2015-01-09','2015-10-10','2015-10-11','2015-10-12','2015-11-10','2015-11-11','2015-11-12','2015-11-25','2015-11-26','2015-11-27','2015-11-28','2015-12-23','2015-12-24','2015-12-25','2015-12-26','2015-12-30','2015-12-31','2015-02-14','2015-02-15','2015-02-16','2015-02-17','2015-05-24','2015-05-25','2015-05-26','2015-07-03','2015-07-04','2015-07-05','2015-09-05','2015-09-06','2015-09-07','2016-01-01','2016-01-16','2016-01-17','2016-01-18','2016-01-02','2016-10-10','2016-10-08','2016-10-09','2016-11-10','2016-11-11','2016-11-12','2016-11-23','2016-11-24','2016-11-25','2016-11-26','2016-12-23','2016-12-24','2016-12-25','2016-12-26','2016-12-30','2016-12-31','2016-02-20','2016-02-21','2016-02-22','2016-05-28','2016-05-29','2016-05-30','2016-07-03','2016-07-04','2016-07-05','2016-09-03','2016-09-04','2016-09-05','2017-01-01','2017-01-02','2017-01-21','2017-01-22','2017-01-23','2017-10-07','2017-10-08','2017-10-09','2017-11-10','2017-11-11','2017-11-12','2017-11-22','2017-11-23','2017-11-24','2017-11-25','2017-12-23','2017-12-24','2017-12-25','2017-12-26','2017-12-30','2017-12-31','2017-02-18','2017-02-19','2017-02-20','2017-05-27','2017-05-28','2017-05-29','2017-07-03','2017-07-04','2017-07-05','2017-09-02','2017-09-03','2017-09-04','2018-01-01','2018-01-02','2018-01-20','2018-01-21','2018-01-22','2018-10-13','2018-10-14','2018-10-15','2018-11-10','2018-11-11','2018-11-12','2018-11-28','2018-11-29','2018-11-30','2018-12-01','2018-12-23','2018-12-24','2018-12-25','2018-12-26','2018-12-30','2018-12-31','2018-02-17','2018-02-18','2018-02-19','2018-05-26','2018-05-27','2018-05-28','2018-07-03','2018-07-04','2018-07-05','2018-09-01','2018-09-02','2018-09-03','2019-01-01','2019-01-02','2019-01-20','2019-01-21','2019-01-22','2019-10-12','2019-10-13','2019-10-14','2019-11-10','2019-11-11','2019-11-12','2019-11-27','2019-11-28','2019-11-29','2019-11-30','2019-12-23','2019-12-24','2019-12-25','2019-12-26','2019-12-30','2019-12-31','2019-02-16','2019-02-17','2019-02-18','2019-05-25','2019-05-26','2019-05-27','2019-07-03','2019-07-04','2019-07-05','2019-08-31','2019-09-01','2019-09-02','2020-01-01','2020-01-18','2020-01-19','2020-01-02','2020-01-20','2020-10-10','2020-10-11','2020-10-12','2020-11-10','2020-11-11','2020-11-12','2020-11-25','2020-11-26','2020-11-27','2020-11-28','2020-12-23','2020-12-24','2020-12-25','2020-12-26','2020-12-30','2020-12-31','2020-02-15','2020-02-16','2020-02-17','2020-05-23','2020-05-24','2020-05-25','2020-07-03','2020-07-04','2020-07-05','2020-09-05','2020-09-06','2020-09-07']

    df = df.withColumn("is_holiday", F.when(F.col("FL_DATE").isin(holiday_list), 1).otherwise(0))


################################################
#### Want to BUCKET into (e.g. 8 hr windows) ###
################################################

    conditions = [
        (F.hour("sched_depart_date_time_UTC").isin([23, 0, 1, 2, 3, 4, 5, 6]), 1),
        (F.hour("sched_depart_date_time_UTC").isin([7, 8, 9, 10, 11, 12, 13, 14]), 2),
        (F.hour("sched_depart_date_time_UTC").isin([15, 16, 17, 18, 19, 20, 21, 22]), 3)]

    df = df.withColumn("part_of_day", F.coalesce(*[F.when(condition, value) for condition, value in conditions]))


    ### % of delayed flights 2 hours prior to flight hour

    # Sort the DataFrame by 'FL_DATE' and 'hour_of_day' in ascending order
    df = df.orderBy("FL_DATE", "hour_of_day")

    # Define a Window specification partitioned by 'FL_DATE' and 'hour_of_day' and ordered by 'FL_DATE' and 'hour_of_day'
    window_spec = Window.partitionBy("FL_DATE", "hour_of_day","ORIGIN").orderBy("FL_DATE", "hour_of_day", "ORIGIN")

    # Calculate the number of delayed flights in the hour that is 2 hours prior to 'hour_of_day'
    df = df.withColumn("DELAYED_FLIGHTS_2_HOURS_PRIOR", F.lag(F.col("DEP_DEL15").cast("integer"), 2).over(window_spec))

    # Calculate the total number of flights in the hour that is 2 hours prior to 'hour_of_day'
    df = df.withColumn("TOTAL_FLIGHTS_2_HOURS_PRIOR", F.lag(F.lit(1)).over(window_spec))

    # Group by 'FL_DATE', 'hour_of_day', and 'departure_airport' and calculate aggregated_data
    aggregated_data = df.groupBy("FL_DATE", "hour_of_day", "ORIGIN").agg(
        F.sum("DELAYED_FLIGHTS_2_HOURS_PRIOR").alias("DELAYED_FLIGHTS_2_HOURS_PRIOR"),
        F.sum("TOTAL_FLIGHTS_2_HOURS_PRIOR").alias("TOTAL_FLIGHTS_2_HOURS_PRIOR")
    )

    # Calculate the percentage of delayed flights
    aggregated_data = aggregated_data.withColumn("DELAYED_PERCENTAGE_2_HOURS_PRIOR",
                                                (F.col("DELAYED_FLIGHTS_2_HOURS_PRIOR") / F.col("TOTAL_FLIGHTS_2_HOURS_PRIOR")) * 100)

    # Join the aggregated_data with 'OTPW_12m_small' on 'FL_DATE', 'hour_of_day', and 'departure_airport'
    df = df.join(aggregated_data, on=["FL_DATE", "hour_of_day", "ORIGIN"], how="left_outer")

    # Clean up null values:
    cols = ['DELAYED_PERCENTAGE_2_HOURS_PRIOR']

    # Replace null values in the specified columns with the desired value
    for c in cols:
        # Replace null values with the value you want (e.g., 0, 'NA', etc.)
        df = df.fillna(0, subset=c)  # Replace nulls with 0. You can use any other value you prefer.

    # Drop unneeded cols:
    columns_to_drop = ['DELAYED_FLIGHTS_2_HOURS_PRIOR', 'TOTAL_FLIGHTS_2_HOURS_PRIOR']

    # Drop the specified columns from the DataFrame 
    df = df.drop(*columns_to_drop)

    return df

# def add_graph_features(df, save_as=None):
#     train_raw = df.filter(col('YEAR') != '2019')

#     edges = (train_raw.select('ORIGIN', 'DEST')
#             .withColumnRenamed("ORIGIN", "src")
#             .withColumnRenamed("DEST", "dst")).distinct()

#     nodes = (edges.distinct().select('src')
#             .union(edges.select('dst'))
#             .distinct()
#             .withColumnRenamed("src", "id"))

#     graph = GraphFrame(nodes, edges)

#     pagerank = spark.read.parquet(f"{blob_url}/pagerank")

#     in_degrees = graph.inDegrees
#     out_degrees = graph.outDegrees
#     result = out_degrees.join(in_degrees, out_degrees.id == in_degrees.id, 'left').drop(in_degrees.id)
#     result = result.orderBy('id', ascending=True).select('id', 'inDegree', 'outDegree').cache()

#     df = OTPW_60M
#     df = df.join(result, df.ORIGIN == result.id, 'left').drop(result.id)
#     df = df.join(pagerank, df.ORIGIN == pagerank.id, 'left').drop(pagerank.id)

#     OTPW_60M.write.option("header", "true").parquet(f"{blob_url}/pipeline_train_js/")
    

def one_hot_encode(df):
    #list of string variables
    categorical_columns = ['FL_DATE','hour_of_day','ORIGIN','QUARTER','DAY_OF_MONTH','DAY_OF_WEEK','YEAR','MONTH','OP_UNIQUE_CARRIER','DEST','origin_type','is_holiday','part_of_day']
    #categorical_columns = [c for c, t in df.dtypes if t == 'string' and c != 'DEP_TIME']
    # Step 1: StringIndexer - Convert categorical string values to numerical indices
    indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep") for col in categorical_columns]
    # Step 2: OneHotEncoder - Convert numerical indices to one-hot encoded vectors
    encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_onehot") for col in categorical_columns]
    # Step 3: Create a pipeline to chain the StringIndexer and OneHotEncoder stages
    pipeline = Pipeline(stages=indexers + encoders)
    # Fit and transform the pipeline on the DataFrame
    encoded_data_df = pipeline.fit(df).transform(df)
    # Select the original columns and the newly created one-hot encoded columns
    selected_columns = df.columns + [col + "_onehot" for col in categorical_columns]
    # Select only the required columns from the DataFrame
    final_df = encoded_data_df.select(selected_columns)
    return final_df

def initial_split(df):


    # Filter out the data for 2019 as test set
    test_data_df = df.filter("YEAR = 2019")
    

    # Filter the rest of the data as training set
    train_data_df = df.filter("YEAR != 2019")

    return train_data_df, test_data_df

def convert_cat_dataT(df):
    # Cast categorical features to string 
    cat_cols = ['FL_DATE','hour_of_day','ORIGIN','QUARTER','DAY_OF_MONTH','DAY_OF_WEEK','YEAR','MONTH','OP_UNIQUE_CARRIER','DEST','origin_type','is_holiday','part_of_day']
    for column_name in cat_cols:
        print('columnName to convert:',column_name)
        df = df.withColumn(column_name, df[column_name].cast(StringType()))
    return df

def down_sample(df):
    total_count = df.count()
    delay_count = df.filter("DEP_DEL15 = 1").count()
    split = .5
    on_time_count = df.filter("DEP_DEL15 = 0").count()

    # Calculate the sampling fraction for the on-time flights
    on_time_sampling_fraction = delay_count / on_time_count

    # Sample the on-time flights with the calculated sampling fraction
    sampled_on_time_flights = df.filter("DEP_DEL15 = 0").sample(withReplacement=False, fraction=on_time_sampling_fraction)

    # Union the sampled delayed and on-time flights to create the downsampled DataFrame
    downsampled_flight_data = df.filter("DEP_DEL15 = 1").union(sampled_on_time_flights)

    # Show the resulting downsampled DataFrame
    # display(downsampled_flight_data.limit(100))
    return downsampled_flight_data

In [0]:
class CustomColumnNormalizer(Transformer):
    def __init__(self, input_cols=None, output_col_suffix="_normalized"):
        self.input_cols = input_cols
        self.output_col_suffix = output_col_suffix
        self.vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="sfeatures")
        self.scaler = MinMaxScaler(inputCol="sfeatures", outputCol=output_col_suffix)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        assembled_data = self.vector_assembler.transform(df)
        normalized_data = self.scaler.fit(assembled_data).transform(assembled_data)
        
        # Get the original columns without the input_cols for renaming
        original_columns = [col for col in df.columns if col not in self.input_cols]
        
        # Create a mapping for renaming the new columns
        rename_mapping = {col: col + self.output_col_suffix if col in self.input_cols else col for col in df.columns}
        
        # Apply alias to DataFrame columns using col() function
        normalized_data = normalized_data.select([col(col_name).alias(rename_mapping.get(col_name, col_name)) for col_name in normalized_data.columns])
        
        # Combine the original columns and the new normalized columns
        combined_data = df.select(original_columns).join(normalized_data, on=original_columns, how='inner')
        
        return combined_data

#### Build Pipeline
##### -- Pipeline only includes data cleansing and transformations
##### -- Cross validation + modeling will be done with the results

In [0]:
# Create custom function transformers for pipeline
num_cols = ['HourlyDewPointTemperature','HourlyDryBulbTemperature','HourlyPrecipitation','HourlyRelativeHumidity','HourlySeaLevelPressure','HourlyVisibility','HourlyWetBulbTemperature','HourlyWindDirection','HourlyWindSpeed','DELAYED_PERCENTAGE_2_HOURS_PRIOR','DISTANCE', 'inDegree', 'outDegree','pagerank']
cleanser = CustomFunctionTransformer(data_cleansing)
feature_engineer = CustomFunctionTransformer(feature_engineering)
ohe = CustomFunctionTransformer(one_hot_encode)
handle_nulls = CustomFunctionTransformer(null_handling)
cat_dataT = CustomFunctionTransformer(convert_cat_dataT)
custom_column_normalizer = CustomColumnNormalizer(input_cols=num_cols, output_col_suffix="scaled_features")
train_test_split = CustomFunctionTransformer(initial_split)

In [0]:
# Data Preparation Pipeline
pipeline = Pipeline(stages=[
    cleanser,
    feature_engineer,
    handle_nulls,
    train_test_split
])

# Run pipeline
pipeline_model = pipeline.fit(OPT_60M_gf)

# Get the transformed data
train, test = pipeline_model.transform(OPT_60M_gf)

columnName to convert: FL_DATE
columnName to convert: hour_of_day
columnName to convert: ORIGIN
columnName to convert: QUARTER
columnName to convert: DAY_OF_MONTH
columnName to convert: DAY_OF_WEEK
columnName to convert: YEAR
columnName to convert: MONTH
columnName to convert: OP_UNIQUE_CARRIER
columnName to convert: DEST
columnName to convert: origin_type
columnName to convert: sched_depart_date_time_UTC
columnName to convert: is_holiday
columnName to convert: part_of_day
('HourlyDewPointTemperature', 'median')
('HourlyDryBulbTemperature', 'median')
('HourlyPrecipitation', 'median')
('HourlyRelativeHumidity', 'median')
('HourlySeaLevelPressure', 'median')
('HourlyVisibility', 'median')
('HourlyWetBulbTemperature', 'median')
('HourlyWindDirection', 'median')
('HourlyWindSpeed', 'median')


In [0]:
print("Train dataset date range:",initial_train.agg(min('FL_DATE'), max('FL_DATE')).first())
print("Test dataset date range:",initial_test.agg(min('FL_DATE'), max('FL_DATE')).first())

Train dataset date range: Row(min(FL_DATE)='2015-01-01', max(FL_DATE)='2018-12-31')
Test dataset date range: Row(min(FL_DATE)='2019-01-01', max(FL_DATE)='2019-12-30')


In [0]:
print("Train dataset size:",initial_train.count())
print("Test dataset size:",initial_test.count())

Train dataset size: 23933364
Test dataset size: 7263966


# Pagerank

In [0]:
train_raw = OTPW_60M.filter(col('YEAR') != '2019')

edges = (train_raw.select('ORIGIN', 'DEST')
        .withColumnRenamed("ORIGIN", "src")
        .withColumnRenamed("DEST", "dst")).distinct()

nodes = (edges.distinct().select('src')
        .union(edges.select('dst'))
        .distinct()
        .withColumnRenamed("src", "id"))

graph = GraphFrame(nodes, edges)

In [0]:
pagerank = spark.read.parquet(f"{blob_url}/pagerank")

in_degrees = graph.inDegrees
out_degrees = graph.outDegrees
result = out_degrees.join(in_degrees, out_degrees.id == in_degrees.id, 'left').drop(in_degrees.id)
result = result

graph_features = pagerank.join(result, pagerank.id == result.id, 'left')
graph_features.orderBy('id', ascending=True).select('id', 'inDegree', 'outDegree').cache()

In [0]:
df = OTPW_60M
df = df.join(result, df.ORIGIN == result.id, 'left').drop(result.id)
df = df.join(pagerank, df.ORIGIN == pagerank.id, 'left').drop(pagerank.id).cache()

In [0]:
df.write.option("header", "true").parquet(f"{blob_url}/pipeline_train_js/")

In [0]:
OPT_60M_gf = spark.read.parquet(f"{blob_url}/pipeline_train_js/")
df = OTPW_60M
df = df.join(result, df.ORIGIN == result.id, 'left').drop(result.id)

In [0]:
# edges
g = GraphFrame(nodes, edges)

In [0]:
nodes_bro = edges.select("ORIGIN").distinct().union(edges.select("DEST").distinct())
origins_df = edges.select("ORIGIN").distinct()
destinations_df = edges.select("DEST").distinct()

vertices_df = origins_df.union(destinations_df).distinct()

edges_df = edges

edges_df = edges.withColumnRenamed("ORIGIN", "src").withColumnRenamed("DEST", "dst")

graph = GraphFrame(vertices_df, edges_df)

results = graph.pageRank(resetProbability=0.15, maxIter=10)

pagerank_scores = results.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False)
pagerank_scores.show()


In [0]:
# def pageranker(df):
origins_df = edges.select("ORIGIN").distinct().withColumnRenamed("origin", "id")

# Create a DataFrame with unique destinations and rename the 'destination' column to 'id'
destinations_df = edges.select("DEST").distinct().withColumnRenamed("destination", "id")

# Combine origins and destinations to create vertices DataFrame
vertices_df = origins_df.union(destinations_df).distinct()

# Create edges DataFrame with 'src' and 'dst' columns representing the origin and destination of each flight
edges_df = edges.withColumnRenamed("origin", "src").withColumnRenamed("DEST", "dst")

graph = GraphFrame(vertices_df, edges_df)

results = graph.pageRank(resetProbability=0.15, maxIter=10)

pagerank_scores = results.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False)
pagerank_scores.show()

In [0]:
def pageranker(df):

    df = df.repartition(100) 

    origins_df = df.select("ORIGIN").distinct().withColumnRenamed("origin", "id")

    # Create a DataFrame with unique destinations and rename the 'destination' column to 'id'
    destinations_df = df.select("DEST").distinct().withColumnRenamed("destination", "id")

    # Combine origins and destinations to create vertices DataFrame
    vertices_df = origins_df.union(destinations_df).distinct()

    # Create edges DataFrame with 'src' and 'dst' columns representing the origin and destination of each flight
    edges_df = df.withColumnRenamed("origin", "src").withColumnRenamed("DEST", "dst")

    graph = GraphFrame(vertices_df, edges_df)

    results = graph.pageRank(resetProbability=0.15, maxIter=10)

    pagerank_scores = results.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False)
    return pagerank_scores

In [0]:
#### Save or Load pagerank results:
# pagerank_60m = test
# pagerank_60m.write.option("header", "true").parquet(f"{blob_url}/pagerank/")
pagerank_60m = spark.read.parquet(f"{blob_url}pagerank/pagerank_60m/")

In [0]:
# Group by the 'ORIGIN' column
grouped_df = df.groupBy("ORIGIN")\
    .agg(
        # Collect unique values in 'inDegree' column
        F.collect_set("inDegree").alias("unique_inDegrees"),
        
        # Collect unique values in 'outDegree' column
        F.collect_set("outDegree").alias("unique_outDegrees")
    )

# Show the result
display(grouped_df)

In [0]:
# use the graphframe
flights_df = OTPW_60M.limit(500)
origins_df = flights_df.select("ORIGIN").distinct()

# Create a DataFrame with unique destinations and rename the 'destination' column to 'id'
destinations_df = flights_df.select("DEST").distinct()

# Combine origins and destinations to create vertices DataFrame
vertices_df = origins_df.union(destinations_df).distinct()

# Create edges DataFrame with 'src' and 'dst' columns representing the origin and destination of each flight
edges_df = flights_df.withColumnRenamed("origin", "src").withColumnRenamed("DEST", "dst")

graph = GraphFrame(vertices_df, edges_df)


flights_df = OTPW_60M.limit(100)
vertices_df = graph.vertices
edges_df = graph.edges
in_degrees = graph.inDegrees
out_degrees = graph.outDegrees


other_columns = []
result_df = flights_df.join(edges_df, (flights_df.origin == edges_df.src) & (flights_df.destination == edges_df.dst))
result_df = result_df.join(in_degrees, result_df.origin == in_degrees.id, 'left_outer').withColumnRenamed("inDegree", "origin_in_degree")
result_df = result_df.join(out_degrees, result_df.origin == out_degrees.id, 'left_outer').withColumnRenamed("outDegree", "origin_out_degree")
result_df = result_df.select("origin", "destination", "origin_in_degree", "origin_out_degree", *other_columns)


# Normalization of Data
#### Normalize Test Data, Hold Off On Normalizing Train/Validation Data until SMOTE function

In [0]:
class CustomColumnNormalizer(Transformer):
    def __init__(self, input_cols=None, scaler_model=None, output_col_suffix="_normalized"):
        self.input_cols = input_cols
        self.output_col_suffix = output_col_suffix
        self.vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="num_features")
        self.scaler_model = scaler_model
    
    def _transform(self, df: DataFrame) -> DataFrame:
        assembled_data = self.vector_assembler.transform(df)
        normalized_data = self.scaler_model.transform(assembled_data)

        return normalized_data

In [0]:
# Create Custom Transformer To Normalize All Datasets (Train/Validation/Test)
class CustomColumnNormalizer(Transformer):
    def __init__(self, input_cols=None, scaler_model=None, output_col_suffix="_normalized"):
        self.input_cols = input_cols
        self.output_col_suffix = output_col_suffix
        self.vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="num_features")
        self.scaler_model = scaler_model
    
    def _transform(self, df: DataFrame) -> DataFrame:
        assembled_data = self.vector_assembler.transform(df)
        normalized_data = self.scaler_model.transform(assembled_data)

        return normalized_data

# Create Scaler Model Using Training data
num_cols = ['HourlyDewPointTemperature','HourlyDryBulbTemperature','HourlyPrecipitation','HourlyRelativeHumidity','HourlySeaLevelPressure','HourlyVisibility','HourlyWetBulbTemperature','HourlyWindDirection','HourlyWindSpeed','DELAYED_PERCENTAGE_2_HOURS_PRIOR','DISTANCE']
cat_cols = ['FL_DATE','hour_of_day','ORIGIN','QUARTER','DAY_OF_MONTH','DAY_OF_WEEK','YEAR','MONTH','OP_UNIQUE_CARRIER','DEST','origin_type','sched_depart_date_time_UTC','is_holiday','part_of_day']
num_assembler = VectorAssembler(inputCols = num_cols, outputCol = 'num_features')
num_assembled_train = num_assembler.transform(initial_train)
train_scaler = MinMaxScaler(inputCol='num_features', outputCol='scaled_features').fit(num_assembled_train)
custom_column_normalizer = CustomColumnNormalizer(input_cols=num_cols, scaler_model=train_scaler, output_col_suffix="scaled_features")

# Scale and OHE Test Data
pipeline_test = Pipeline(stages=[custom_column_normalizer,cat_dataT,ohe])
test_vectorized = pipeline_test.fit(initial_test).transform(initial_test)
test_vectorized = test_vectorized.withColumnRenamed('DEP_DEL15', 'label').drop('num_features')

columnName to convert: FL_DATE
columnName to convert: hour_of_day
columnName to convert: ORIGIN
columnName to convert: QUARTER
columnName to convert: DAY_OF_MONTH
columnName to convert: DAY_OF_WEEK
columnName to convert: YEAR
columnName to convert: MONTH
columnName to convert: OP_UNIQUE_CARRIER
columnName to convert: DEST
columnName to convert: origin_type
columnName to convert: sched_depart_date_time_UTC
columnName to convert: is_holiday
columnName to convert: part_of_day
categorical_columns ['FL_DATE', 'hour_of_day', 'ORIGIN', 'QUARTER', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'YEAR', 'MONTH', 'OP_UNIQUE_CARRIER', 'DEST', 'origin_type', 'sched_depart_date_time_UTC', 'is_holiday', 'part_of_day']
dtypes: [('FL_DATE', 'string'), ('hour_of_day', 'string'), ('ORIGIN', 'string'), ('QUARTER', 'string'), ('DAY_OF_MONTH', 'string'), ('DAY_OF_WEEK', 'string'), ('YEAR', 'string'), ('MONTH', 'string'), ('OP_UNIQUE_CARRIER', 'string'), ('DEST', 'string'), ('DEP_TIME', 'string'), ('DEP_DEL15', 'float'), ('

# Cross Validation Split On Train Data

In [0]:
BlockSplit = namedtuple("BlockSplit", ["min_date", "train_cut", "val_cut"])

def make_block_splits(min_date, max_date, train_width, val_width):
    """
    Return a list of breakpoints for building the train and validation folds.

    Each item in the list contains a namedtuple of dates: (min_date, train_cut, val_cut).
    The folds can be generated using these cuts as follows: 
    
    * train data is (min_date <= flight_date < train_cut)
    * val data is (train_cut <= flight_date < val_cut)

    Args:
        min_date - the first date to include in the first fold
        max_date - the validation cut of the last fold (i.e. this is the first day
            of the test set and is not included in the folds)
        train_width - the timespan covered by each training set
        val_delta - the timespan covered by each validation set

    Returns:
        a list of ``BlockSplit`` instances
    """
    blocks = list()
    train_min = min_date
    val_cut = min_date
    
    # Loop over and create blocks
    while True:
        train_cut = train_min + train_width
        val_cut = train_cut + val_width
        if train_cut > max_date or val_cut > max_date:
            break
        blocks.append(BlockSplit(train_min, train_cut, val_cut))
        train_min = val_cut
    
    return blocks

def make_folds(df, date_col, splits):
    """
    Make folds using the specified block splits.

    Args:
        df - the dataframe to make folds from
        date_col - the name of a date column to split on
        splits - a list of ``BlockSplit`` instances
    
    Returns:
        a list of (train_df, val_df) tuples
    """
    folds = list()
    count = 0
    for split in splits:
        print("FOLD",count)
        train_df = df.filter((df[date_col] >= split.min_date) & (df[date_col] < split.train_cut))
        train_df = down_sample(train_df)
        val_df = df.filter((df[date_col] >= split.train_cut) & (df[date_col] < split.val_cut))
        folds.append((train_df, val_df))
        count += 1
    return folds

def save_folds_to_blob(folds, blob_url, fold_name):
    for i, (train_df, val_df) in enumerate(folds):
        dbutils.fs.rm(f"{blob_url}/{fold_name}/train_{i}_df", recurse=True)
        dbutils.fs.rm(f"{blob_url}/{fold_name}/val_{i}_df", recurse=True)
        train_df.write.option("header", "true").parquet(f"{blob_url}/{fold_name}/train_{i}_df")
        val_df.write.option("header", "true").parquet(f"{blob_url}/{fold_name}/val_{i}_df")

def make_folds_and_save(df_in, splits, fold_name="folds"):
    """
    Create training folds and save to blob storage.

    Args:
        df_in - the dataframe to split into folds
        splits - the block splits as returned by make_block_splits()
    """
    if input(f"Are you sure you want to overwrite '{fold_name}'? (y/n)").lower().strip() == "y":
        print("Making train/val folds...")
        df = df_in
        folds = make_folds(df, "FL_DATE", splits)
        save_folds_to_blob(folds, blob_url, fold_name)
    else:
        print("Cancelled")

def save_df(df_in):
    """
    Create training folds and save to blob storage.

    Args:
        df_in - the dataframe to split into folds
        splits - the block splits as returned by make_block_splits()
    """
    if input(f"Are you sure you want to overwrite '{fold_name}'? (y/n)").lower().strip() == "y":
        print("Making train/val folds...")
        df = df_in
        folds = make_folds(df, "FL_DATE", splits)
        save_folds_to_blob(folds, blob_url, fold_name)
    else:
        print("Cancelled")
        

In [0]:
dbutils.fs.rm(f"wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/", recurse=True)

False

In [0]:
blob_container  = "w261-container"
storage_account = "whiskeythedog"
secret_scope    = "whiskeythedog-scope"                   
secret_key      = "whikeythedog-key"  
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

# Compute the split points. (This does not actually create the folds)
splits = make_block_splits(
    min_date = date(2015,1,1), 
    max_date = date(2018,12,31), 
    train_width = timedelta(days=250), 
    val_width = timedelta(days=40),
)

for i, split in enumerate(splits):
    print(f"Fold {i} Splits:")
    print(f"  Train: {split.min_date} <= x < {split.train_cut}")
    print(f"    Val: {split.train_cut} <= x < {split.val_cut}")

make_folds_and_save(initial_train, splits, fold_name="temp_folds")

Fold 0 Splits:
  Train: 2015-01-01 <= x < 2015-09-08
    Val: 2015-09-08 <= x < 2015-10-18
Fold 1 Splits:
  Train: 2015-10-18 <= x < 2016-06-24
    Val: 2016-06-24 <= x < 2016-08-03
Fold 2 Splits:
  Train: 2016-08-03 <= x < 2017-04-10
    Val: 2017-04-10 <= x < 2017-05-20
Fold 3 Splits:
  Train: 2017-05-20 <= x < 2018-01-25
    Val: 2018-01-25 <= x < 2018-03-06
Fold 4 Splits:
  Train: 2018-03-06 <= x < 2018-11-11
    Val: 2018-11-11 <= x < 2018-12-21


Are you sure you want to overwrite 'temp_folds'? (y/n) y

Making train/val folds...
FOLD 0
FOLD 1
FOLD 2
FOLD 3
FOLD 4


In [0]:
display(dbutils.fs.ls(f"{blob_url}/temp_folds/"))

path,name,size,modificationTime
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/train_0_df/,train_0_df/,0,1691353388000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/train_1_df/,train_1_df/,0,1691353543000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/train_2_df/,train_2_df/,0,1691353627000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/train_3_df/,train_3_df/,0,1691353696000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/train_4_df/,train_4_df/,0,1691353767000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/val_0_df/,val_0_df/,0,1691353479000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/val_1_df/,val_1_df/,0,1691353589000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/val_2_df/,val_2_df/,0,1691353656000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/val_3_df/,val_3_df/,0,1691353729000
wasbs://w261-container@whiskeythedog.blob.core.windows.net/temp_folds/val_4_df/,val_4_df/,0,1691353801000


# SMOTE

In [0]:
def pre_smote_df_process(df,num_cols,cat_cols,target_col,index_suffix="_index"):
    '''
    string indexer and vector assembler
    inputs:
    * df: spark df, original
    * num_cols: numerical cols to be assembled
    * cat_cols: categorical cols to be stringindexed
    * target_col: prediction target
    * index_suffix: will be the suffix after string indexing
    output:
    * vectorized: spark df, after stringindex and vector assemble, ready for smote
    '''
    print('pre smote process')
    if(df.select(target_col).distinct().count() != 2):
        raise ValueError("Target col must have exactly 2 classes")

    if target_col in num_cols:
        num_cols.remove(target_col)

    # window = Window.orderBy(F.monotonically_increasing_id())
    # df = df.withColumn("index", F.row_number().over(window))  

    # # Remove DEP_TIME column (will not be part of feature set)
    # # To be reinserted later
    # removed_dep_time = "DEP_TIME"
    # removed_dep_time_values = df.select("index", removed_dep_time)
    # df = df.drop('DEP_TIME')

    # # Remove DEP_DEL15 label (will not be part of feature set)
    # # To be reinserted later
    # removed_dep_del15 = "DEP_DEL15"
    # removed_dep_del15_values = df.select("index", removed_dep_del15)
    # df = df.drop('DEP_DEL15')

    # index the string cols, except possibly for the label col
    # index the string cols, except possibly for the label col 
    cat_indexers = [(column, StringIndexer(inputCol=column, outputCol=column+index_suffix, handleInvalid='keep').fit(df)) for column in list(set(cat_cols)-set([target_col]))]
    assemble_stages = [x[1] for x in cat_indexers]
    # add the stage of numerical vector assembler
    assembler = VectorAssembler(inputCols = num_cols, outputCol = 'features')
    #assemble_stages.append(custom_column_normalizer)
    assemble_stages.append(assembler)
    pipeline = Pipeline(stages=assemble_stages)
    pos_vectorized = pipeline.fit(df).transform(df)

    # # add the stage of numerical vector assembler
    # assemble_stages.append(custom_column_normalizer)
    # pipeline = Pipeline(stages=assemble_stages)
    # pos_vectorized = pipeline.fit(df).transform(df)

    # drop original num cols and cat cols
    drop_cols = num_cols+cat_cols
    
    keep_cols = [a for a in pos_vectorized.columns if a not in drop_cols]
    
    vectorized = pos_vectorized.select(*keep_cols).withColumn('label',pos_vectorized[target_col]).drop(target_col)
    
    return vectorized, cat_indexers

def smote(vectorized_sdf,smote_config):
    '''
    contains logic to perform smote oversampling, given a spark df with 2 classes
    inputs:
    * vectorized_sdf: cat cols are already stringindexed, num cols are assembled into 'features' vector
      df target col should be 'label'
    * smote_config: config obj containing smote parameters
    output:
    * oversampled_df: spark df after smote oversampling
    '''
    dataInput_min = vectorized_sdf[vectorized_sdf['label'] == 1]
    dataInput_maj = vectorized_sdf[vectorized_sdf['label'] == 0]
    
    # LSH, bucketed random projection
    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",seed=smote_config['seed'], bucketLength=smote_config['bucketLength'])
    # smote only applies on existing minority instances    
    model = brp.fit(dataInput_min)
    model.transform(dataInput_min)

    # here distance is calculated from brp's param inputCol
    self_join_w_distance = model.approxSimilarityJoin(dataInput_min, dataInput_min, float("inf"), distCol="EuclideanDistance")

    # remove self-comparison (distance 0)
    self_join_w_distance = self_join_w_distance.filter(self_join_w_distance.EuclideanDistance > 0)

    over_original_rows = Window.partitionBy("datasetA").orderBy("EuclideanDistance")

    self_similarity_df = self_join_w_distance.withColumn("r_num", F.row_number().over(over_original_rows))

    self_similarity_df_selected = self_similarity_df.filter(self_similarity_df.r_num <= smote_config['k'])

    over_original_rows_no_order = Window.partitionBy('datasetA')

    # list to store batches of synthetic data
    res = []
    
    # two udf for vector add and subtract, subtraction include a random factor [0,1]
    subtract_vector_udf = F.udf(lambda arr: random.uniform(0, 1)*(arr[0]-arr[1]), VectorUDT())
    add_vector_udf = F.udf(lambda arr: arr[0]+arr[1], VectorUDT())
    
    # retain original columns
    original_cols = dataInput_min.columns
    
    for i in range(smote_config['multiplier']):
        print("generating batch %s of synthetic instances"%i)
        # logic to randomly select neighbour: pick the largest random number generated row as the neighbour
        df_random_sel = self_similarity_df_selected.withColumn("rand", F.rand()).withColumn('max_rand', F.max('rand').over(over_original_rows_no_order))\
                            .where(F.col('rand') == F.col('max_rand')).drop(*['max_rand','rand','r_num'])
        # create synthetic feature numerical part
        df_vec_diff = df_random_sel.select('*', subtract_vector_udf(F.array('datasetA.features', 'datasetB.features')).alias('vec_diff'))
        df_vec_modified = df_vec_diff.select('*', add_vector_udf(F.array('datasetA.features', 'vec_diff')).alias('features'))
        
        # for categorical cols, either pick original or the neighbour's cat values
        for c in original_cols:
            # randomly select neighbour or original data
            col_sub = random.choice(['datasetA','datasetB'])
            val = "{0}.{1}".format(col_sub,c)
            if c != 'features':
                # do not unpack original numerical features
                df_vec_modified = df_vec_modified.withColumn(c,F.col(val))
        
        # this df_vec_modified is the synthetic minority instances,
        df_vec_modified = df_vec_modified.drop(*['datasetA','datasetB','vec_diff','EuclideanDistance'])
        
        res.append(df_vec_modified)
    
    dfunion = reduce(DataFrame.unionAll, res)
    # union synthetic instances with original full (both minority and majority) df
    oversampled_df = dfunion.union(vectorized_sdf.select(dfunion.columns))
    
    return oversampled_df
def reverse_string_index(df, indexers):
    print('reverse string index')

    for i in indexers:
        current_col = i[0] + '_smoteIndex'
        og_col = i[0]
        string_indexer_model = i[1]
        mapping = dict(enumerate(string_indexer_model.labels))
        index_to_string = IndexToString(inputCol=current_col, outputCol=og_col, labels=list(mapping.values()))
        # Apply the transformer on the indexed DataFrame
        df = index_to_string.transform(df).drop(current_col)

    return df

def train_smote(train):
    smote_vectorized, cat_indexers = pre_smote_df_process(train,num_cols,cat_cols,'DEP_DEL15',index_suffix="_smoteIndex")
    smote_config = {'seed':42, 'bucketLength':10, 'k':5, 'multiplier':15}
    smote_result = smote(smote_vectorized,smote_config)
    reversed_df = reverse_string_index(smote_result, cat_indexers)
    return reversed_df.cache()

In [0]:
def load_folds_from_blob_and_cache(blob_url, fold_name):
    folds = list()

    # Compute the fold count
    files = dbutils.fs.ls(f"{blob_url}/{fold_name}")
    fold_names = sorted([f.name for f in files if f.name.startswith("train")])
    match = re.match(r"train_(\d+)_df", fold_names[-1])
    fold_count = int(match.group(1)) + 1
    print(f"Loading {fold_count} folds...")

    # Load folds
    for i in range(fold_count):
        train_df = (
            spark.read.parquet(f"{blob_url}/{fold_name}/train_{i}_df")
            .cache()
        )
        val_df = (
            spark.read.parquet(f"{blob_url}/{fold_name}/val_{i}_df")
            .cache()
        )
        folds.append((i, train_df, val_df))
    return folds

In [0]:
    val_i_df = spark.read.parquet(f"{blob_url}/temp_folds/val_{i}_df")
    print('val_i_df count:',val_i_df.count())
    print('val_i_df cols',val_i_df.dtypes)
    pipeline_val = Pipeline(stages=[custom_column_normalizer,cat_dataT])
    print('pipeline val')
    val_i_vectorized = pipeline_val.fit(val_i_df).transform(val_i_df)
    print('fit+transform val')
    val_i_vectorized = val_i_vectorized.withColumnRenamed('DEP_DEL15', 'label').drop('num_features')
    # val_i_vectorized = one_hot_encode(val_i_vectorized)
    print('format val')

In [0]:
train_folds = []
num_folds = int(len(dbutils.fs.ls(f"{blob_url}/temp_folds/"))/2)

for i in range(num_folds): 
    print("FOLD",i)

    train_i_df = spark.read.parquet(f"{blob_url}/temp_folds/train_{i}_df").cache()
    print('train_i_df count:',train_i_df.count())
    print(type(train_i_df))
    print('train_icols',train_i_df.dtypes)
    smote_train_i_df = train_smote(train_i_df)
    print('SMOTE FINISHED!')
    print('SMOTE train_i_df count:',smote_train_i_df.count())
    print('SMOTE train_i_df cols',smote_train_i_df.dtypes)
    train_folds.append((i, smote_train_i_df))
    print("*" * 100)
    print("\n\n")

FOLD 0
train_i_df count: 3952323
<class 'pyspark.sql.dataframe.DataFrame'>
train_icols [('FL_DATE', 'string'), ('hour_of_day', 'int'), ('ORIGIN', 'string'), ('QUARTER', 'string'), ('DAY_OF_MONTH', 'string'), ('DAY_OF_WEEK', 'string'), ('YEAR', 'string'), ('MONTH', 'string'), ('OP_UNIQUE_CARRIER', 'string'), ('DEST', 'string'), ('DEP_TIME', 'string'), ('DEP_DEL15', 'float'), ('DISTANCE', 'float'), ('origin_type', 'string'), ('sched_depart_date_time_UTC', 'string'), ('HourlyDewPointTemperature', 'double'), ('HourlyDryBulbTemperature', 'double'), ('HourlyPrecipitation', 'double'), ('HourlyRelativeHumidity', 'double'), ('HourlySeaLevelPressure', 'double'), ('HourlyVisibility', 'double'), ('HourlyWetBulbTemperature', 'double'), ('HourlyWindDirection', 'double'), ('HourlyWindSpeed', 'double'), ('is_holiday', 'int'), ('part_of_day', 'int'), ('DELAYED_PERCENTAGE_2_HOURS_PRIOR', 'double')]
pre smote process
generating batch 0 of synthetic instances
generating batch 1 of synthetic instances
gene